# Breaking into error mitigation with Mitiq's calibration module

<img src="../_thumbnails/calibration.png" width="400">

This tutorial helps answer the question: "What quantum error
mitigation technique should I use for my problem?". The newly introduced
`mitiq.calibration` module helps answer that in an optimized way, thrhough `Benchmarks` and `Strategies`.

More specifically, this tutorial covers:

- Getting started with Mitiq's calibration module with ZNE
- Use Qiskit noisy simulator with `FakeJakarta` as backend
- Run calibration with some special settings, `RBSettings`, using the `cal.run(log=True)` option

## Getting started with Mitiq

In [1]:
from mitiq.benchmarks import generate_rb_circuits, generate_ghz_circuit, generate_w_circuit
from mitiq.zne import execute_with_zne
from mitiq.zne.scaling import (
    fold_gates_at_random,
    fold_global,
)
from mitiq import (
    Calibrator,
    Settings,
    execute_with_mitigation,
    MeasurementResult,
)
from mitiq.interface.mitiq_qiskit.qiskit_utils import initialized_depolarizing_noise
import qiskit

### Define the circuit to study

#### Global variables

Define global variables for the quantum circuit of interest: number of qubits, depth of the quantum circuit and number of shots.

In [2]:
n_qubits = 12
shots = 10 ** 4

#### Quantum circuit: Randomized benchmarking (RB)

We now use Mitiq's built-in `generate_rb_circuits` from the `mitiq.benchmarks` module to define the quantum circuit.

In [3]:
# circuit = generate_rb_circuits(n_qubits, depth_circuit,return_type="qiskit")[0]
# circuit.measure_all()
# print(len(circuit))
circuit = generate_ghz_circuit(n_qubits, return_type="qiskit")
# circuit = sequence.compose(sequence.inverse())
# circuit = circuit.compose(circuit)
circuit.measure_all()
print(circuit)

         ┌───┐                                                        ░ ┌─┐   »
    q_0: ┤ H ├──■─────────────────────────────────────────────────────░─┤M├───»
         └───┘┌─┴─┐                                                   ░ └╥┘┌─┐»
    q_1: ─────┤ X ├──■────────────────────────────────────────────────░──╫─┤M├»
              └───┘┌─┴─┐                                              ░  ║ └╥┘»
    q_2: ──────────┤ X ├──■───────────────────────────────────────────░──╫──╫─»
                   └───┘┌─┴─┐                                         ░  ║  ║ »
    q_3: ───────────────┤ X ├──■──────────────────────────────────────░──╫──╫─»
                        └───┘┌─┴─┐                                    ░  ║  ║ »
    q_4: ────────────────────┤ X ├──■─────────────────────────────────░──╫──╫─»
                             └───┘┌─┴─┐                               ░  ║  ║ »
    q_5: ─────────────────────────┤ X ├──■────────────────────────────░──╫──╫─»
                                  └───┘┌

We define a function that executes the quantum circuits and returns the expectation value. This is consumed by Mitiq's `execute_with_zne`. In this example, the expectation value is the probability of measuring the ground state, which is what one would expect from an ideal randomized benchmarking circuit.

In [4]:
def execute_circuit(circuit):
    """Execute the input circuit and return the expectation value of |00..0><00..0|"""
    noise_model = initialized_depolarizing_noise(noise_level=0.05) 
    job = qiskit.execute(
            experiments=circuit,
            backend=qiskit.Aer.get_backend("qasm_simulator"),
            noise_model=noise_model,
            basis_gates=noise_model.basis_gates,
            optimization_level=0,  # Important to preserve folded gates.
            shots=shots,
        )

    # Convert from raw measurement counts to the expectation value
    counts = job.result().get_counts()
    if counts.get(n_qubits * "0") is None:
        noisy_expectation_value = 0.
    else:
        noisy_expectation_value = counts.get(n_qubits * "0") / shots
    return noisy_expectation_value

In [5]:
mitigated = execute_with_zne(circuit, execute_circuit, scale_noise=fold_global)
unmitigated = execute_circuit(circuit)
ideal = 1 #property of W circuits

print("ideal = \t \t",ideal)
print("unmitigated = \t \t", unmitigated)
print("mitigated = \t \t", mitigated)

ideal = 	 	 1
unmitigated = 	 	 0.3286
mitigated = 	 	 0.46649999999999997


## Using calibration to improve the results

Let's consider a noisy backend using the Qiskit noisy simulator, `FakeJakarta`. Note that the executor passed to the `Calibrator` object must return counts, as opposed to expectation values.

In [6]:
def execute_calibration(qiskit_circuit):
    """Execute the input circuits and return the measurement results."""
    noise_model = initialized_depolarizing_noise(noise_level=0.02) 
    job = qiskit.execute(
            experiments=circuit,
            backend=qiskit.Aer.get_backend("qasm_simulator"),
            noise_model=noise_model,
            basis_gates=noise_model.basis_gates,
            optimization_level=0,  # Important to preserve folded gates.
            shots=shots,
        )
    # Convert from raw measurement counts to the expectation value
    noisy_counts = job.result().get_counts()
    noisy_counts = { k.replace(" ",""):v for k, v in noisy_counts.items()}
    measurements = MeasurementResult.from_counts(noisy_counts)
    return measurements

We import from the calibration module the key ingredients to use `mitiq.calibration`: the `Calibrator` class, the `mitiq.calibration.settings.Settings` class and the `execute_with_mitigation` function.

Currently `mitiq.calibration` supports ZNE as a technique to calibrate from, tuning different scale factors, extrapolation methods and circuit scaling methods.

Let's run the calibration using an ad-hoc RBSettings and using the `log=True` option in order to print the list of experiments run.

- benchmarks: Circuit type: "rb"
- strategies: use various "zne" strategies, testing various "scale_noise" methods (such as `mitiq.zne.scaling.folding.fold_global` and `mitiq.zne.scaling.folding.fold_gates_at_random`), and ZNE factories for extrapolation (such as `mitiq.zne.inference.RichardsonFactory` and `mitiq.zne.inference.LinearFactory`)

In [7]:
from mitiq.zne.inference import LinearFactory, RichardsonFactory, ExpFactory, PolyFactory

GHZSettings = Settings(
    benchmarks=[
        {
            "circuit_type": "ghz",
            "num_qubits": 10,
        },
    ],
   strategies=[
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 3.0, 5.0]),
        },
    ],
)

In [8]:
cal = Calibrator(execute_calibration, frontend="qiskit", settings=GHZSettings)
cal.run(log=True)

| performance | circuit | method | extrapolation | scale factors | scale_method         |
| ----------- | ------- | ------ | ------------- | ------------- | -------------------- |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_global          |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Richardson    | 1.0, 3.0, 5.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 2.0, 3.0 | fold_gates_at_random |
|     ❌      |   ghz   |  ZNE   | Linear        | 1.0, 3.0, 5.0 | fold_gates_at_random |


As you can see above, several experiments were run, and each one has either a red cross (❌) or a green check (✅) to signal whether the error mitigation experiment obtained an expectation value that is better than the non-mitigated one.

In [9]:
calibrated_mitigated=execute_with_mitigation(circuit, execute_circuit, calibrator=cal)
mitigated=execute_with_zne(circuit, execute_circuit, scale_noise=fold_global)
unmitigated=execute_circuit(circuit)

print("ideal = \t \t",ideal)
print("unmitigated = \t \t",unmitigated)
print("mitigated = \t \t",mitigated)
print("calibrated_mitigated = \t",calibrated_mitigated)

ideal = 	 	 1
unmitigated = 	 	 0.3341
mitigated = 	 	 0.4959999999999998
calibrated_mitigated = 	 0.5175000000000002
